# Libraries and Dependencies

In [1]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
import re

# Interactive mapping library
import folium
from folium import plugins

# Geospatial data processing
import geopandas as gpd
from shapely.geometry import Point

# Visualization utilities
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Data Processing and Geospatial Analysis

In [3]:
# Load municipalities dataset
df_municipios = pd.read_csv('Datasets/Directorio_de_municipios_y_alcaldías_-_DEPARTAMENTO_DE_BOYACÁ_20250919.csv')

# Basic dataset information
print(f"Dataset shape: {df_municipios.shape}")
print(f"Columns: {df_municipios.columns.tolist()}")

# Data cleaning and preprocessing
## Clean municipality names and remove special characters
df_municipios['MUNICIPIO_CLEAN'] = df_municipios['MUNICIPIO'].str.strip()

## Extract coordinates from UBICACION column
def extract_coordinates(location_str):
    """Extract latitude and longitude from POINT string format"""
    if pd.isna(location_str) or location_str == '':
        return None, None
    
    # Use regex to extract coordinates from POINT format
    pattern = r'POINT \(([+-]?\d+\.?\d*) ([+-]?\d+\.?\d*)\)'
    match = re.search(pattern, str(location_str))
    
    if match:
        longitude = float(match.group(1))
        latitude = float(match.group(2))
        return latitude, longitude
    return None, None

# Apply coordinate extraction
coordinates = df_municipios['UBICACION (Fuente DANE)'].apply(extract_coordinates)
df_municipios['latitude'] = [coord[0] for coord in coordinates]
df_municipios['longitude'] = [coord[1] for coord in coordinates]

# Remove rows with missing coordinates
df_municipios_clean = df_municipios.dropna(subset=['latitude', 'longitude']).copy()

# Provincial analysis
province_counts = df_municipios_clean['PROVINCIA'].value_counts()
print(f"\nMunicipalities by Province:")
print(province_counts)

# Altitude analysis
altitude_stats = df_municipios_clean['ALTURA (msnm)'].describe()
print(f"\nAltitude statistics (meters above sea level):")
print(altitude_stats)

# Create altitude categories
def categorize_altitude(altitude):
    """Categorize municipalities by altitude"""
    if altitude < 1000:
        return 'Low (< 1000m)'
    elif altitude < 2000:
        return 'Medium (1000-2000m)'
    elif altitude < 3000:
        return 'High (2000-3000m)'
    else:
        return 'Very High (> 3000m)'

df_municipios_clean['altitude_category'] = df_municipios_clean['ALTURA (msnm)'].apply(categorize_altitude)
altitude_distribution = df_municipios_clean['altitude_category'].value_counts()
print(f"\nAltitude distribution:")
print(altitude_distribution)

# Calculate Boyacá geographic center for map centering
center_lat = df_municipios_clean['latitude'].mean()
center_lon = df_municipios_clean['longitude'].mean()

print(f"\nBoyacá geographic center: {center_lat:.4f}, {center_lon:.4f}")
print(f"Total municipalities with valid coordinates: {len(df_municipios_clean)}")

# Display processed data
df_municipios_clean[['MUNICIPIO', 'PROVINCIA', 'ALTURA (msnm)', 'altitude_category', 'latitude', 'longitude']].head(10)

Dataset shape: (123, 11)
Columns: ['CODIGO DANE MUNICIPIO', 'PROVINCIA', 'MUNICIPIO', 'NIT ALCALDÍA', 'NOMBRE DEL ALCALDE', 'DIRECCIÓN', 'PAGINA WEB', 'CORREOELECTRÓNICO', 'TELEFONO', 'ALTURA (msnm)', 'UBICACION (Fuente DANE)']

Municipalities by Province:
PROVINCIA
OCCIDENTE      16
CENTRO         15
SUGAMUXI       13
RICAURTE       13
MÁRQUEZ        10
TUNDAMA         9
NORTE           9
ORIENTE         8
VALDERRAMA      7
GUTIERREZ       7
LENGUPÁ         6
NEIRA           6
LA LIBERTAD     4
Name: count, dtype: int64

Altitude statistics (meters above sea level):
count     123.000000
mean     2138.552846
std       658.333455
min       100.000000
25%      1700.000000
50%      2360.000000
75%      2627.500000
max      3050.000000
Name: ALTURA (msnm), dtype: float64

Altitude distribution:
altitude_category
High (2000-3000m)      77
Medium (1000-2000m)    34
Low (< 1000m)           9
Very High (> 3000m)     3
Name: count, dtype: int64

Boyacá geographic center: 5.6666, -73.2085
Total 

,MUNICIPIO,PROVINCIA,ALTURA (msnm),altitude_category,latitude,longitude
0,ALMEIDA,ORIENTE,1918,Medium (1000-2000m),4.970857,-73.378933
1,AQUITANIA,SUGAMUXI,3050,Very High (> 3000m),5.518604,-72.883985
2,ARCABUCO,RICAURTE,2739,High (2000-3000m),5.755184,-73.438185
3,BELÉN,TUNDAMA,2650,High (2000-3000m),5.989278,-72.911620
4,BERBEO,LENGUPÁ,1325,Medium (1000-2000m),5.227287,-73.127032
5,BETEITIVA,VALDERRAMA,2600,High (2000-3000m),5.909908,-72.809031
6,BOAVITA,NORTE,2162,High (2000-3000m),6.330653,-72.584959
7,BOYACÁ,MÁRQUEZ,2412,High (2000-3000m),5.454237,-73.362303
8,BRICEÑO,OCCIDENTE,1500,Medium (1000-2000m),5.690879,-73.923260
9,BUENAVISTA,OCCIDENTE,2340,High (2000-3000m),5.512431,-73.942305


# Interactive Maps and Visualizations

In [5]:
# 1. Main interactive map of Boyacá municipalities
# Create base map centered on Boyacá
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=9,
    tiles='OpenStreetMap'
)

# Define colors for each province
province_colors = {
    'CENTRO': '#FF6B6B',
    'NORTE': '#4ECDC4', 
    'OCCIDENTE': '#45B7D1',
    'ORIENTE': '#96CEB4',
    'SUGAMUXI': '#FFEAA7',
    'TUNDAMA': '#DDA0DD',
    'VALDERRAMA': '#98D8C8',
    'RICAURTE': '#FFB347',
    'MÁRQUEZ': '#87CEEB',
    'LENGUPÁ': '#F7DC6F',
    'NEIRA': '#BB8FCE',
    'GUTIERREZ': '#85C1E9',
    'LA LIBERTAD': '#F8C471'
}

# Add markers for each municipality
for idx, row in df_municipios_clean.iterrows():
    # Get province color, default to gray if not found
    color = province_colors.get(row['PROVINCIA'], '#808080')
    
    # Create popup content with municipality information
    popup_content = f"""
    <div style="width: 250px;">
        <h4><b>{row['MUNICIPIO']}</b></h4>
        <hr>
        <b>Province:</b> {row['PROVINCIA']}<br>
        <b>Mayor:</b> {row['NOMBRE DEL ALCALDE']}<br>
        <b>Altitude:</b> {row['ALTURA (msnm)']} meters<br>
        <b>Category:</b> {row['altitude_category']}<br>
        <b>Address:</b> {row['DIRECCIÓN']}<br>
        <b>Phone:</b> {row['TELEFONO']}<br>
        <b>Website:</b> <a href="{row['PAGINA WEB']}" target="_blank">Visit</a><br>
        <b>Email:</b> {row['CORREOELECTRÓNICO']}<br>
        <b>DANE Code:</b> {row['CODIGO DANE MUNICIPIO']}
    </div>
    """
    
    # Add marker to map
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=8,
        popup=folium.Popup(popup_content, max_width=300),
        tooltip=f"{row['MUNICIPIO']} - {row['PROVINCIA']}",
        color='white',
        fillColor=color,
        fillOpacity=0.7,
        weight=2
    ).add_to(m)

# Add a title to the map
title_html = '''
<h3 align="center" style="font-size:20px; color:#2E4057; margin-top:10px;">
<b>Municipalities of Boyacá Department, Colombia</b>
</h3>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Display the map
m

In [ ]:
# 2. Altitude heatmap of Boyacá municipalities
# Create map for altitude visualization
altitude_map = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=9,
    tiles='OpenStreetMap'
)

# Prepare data for heatmap (latitude, longitude, altitude)
heat_data = []
for idx, row in df_municipios_clean.iterrows():
    # Normalize altitude for better visualization (scale 0-1)
    normalized_altitude = row['ALTURA (msnm)'] / df_municipios_clean['ALTURA (msnm)'].max()
    heat_data.append([row['latitude'], row['longitude'], normalized_altitude])

# Add heatmap layer
from folium.plugins import HeatMap
HeatMap(
    heat_data,
    min_opacity=0.3,
    radius=20,
    blur=15,
    max_zoom=1,
    gradient={
        0.0: 'green',
        0.3: 'yellow', 
        0.6: 'orange',
        1.0: 'red'
    }
).add_to(altitude_map)

# Add municipality markers with altitude information
for idx, row in df_municipios_clean.iterrows():
    # Color based on altitude category
    if row['ALTURA (msnm)'] < 1000:
        marker_color = 'green'
    elif row['ALTURA (msnm)'] < 2000:
        marker_color = 'yellow'
    elif row['ALTURA (msnm)'] < 3000:
        marker_color = 'orange'
    else:
        marker_color = 'red'
    
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=6,
        popup=f"<b>{row['MUNICIPIO']}</b><br>Altitude: {row['ALTURA (msnm)']} m",
        tooltip=f"{row['MUNICIPIO']}: {row['ALTURA (msnm)']} m",
        color='white',
        fillColor=marker_color,
        fillOpacity=0.8,
        weight=1
    ).add_to(altitude_map)

# Add title and legend
title_html = '''
<h3 align="center" style="font-size:18px; color:#2E4057; margin-top:10px;">
<b>Altitude Distribution - Boyacá Municipalities</b>
</h3>
<div style="position: fixed; 
            bottom: 50px; right: 50px; width: 180px; height: 120px; 
            background-color: white; border:2px solid grey; z-index:9999; 
            font-size:14px; padding: 10px">
<h4>Altitude Legend</h4>
<i class="fa fa-circle" style="color:green"></i> Low (< 1000m)<br>
<i class="fa fa-circle" style="color:yellow"></i> Medium (1000-2000m)<br>
<i class="fa fa-circle" style="color:orange"></i> High (2000-3000m)<br>
<i class="fa fa-circle" style="color:red"></i> Very High (> 3000m)
</div>
'''
altitude_map.get_root().html.add_child(folium.Element(title_html))

# Display altitude map
altitude_map

In [6]:
# 3. Provincial clustering map with interactive layers
cluster_map = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=8,
    tiles='OpenStreetMap'
)

# Create feature groups for each province
province_groups = {}
for province in df_municipios_clean['PROVINCIA'].unique():
    province_groups[province] = folium.FeatureGroup(name=f"Province: {province}")

# Add municipalities to their respective province groups
for idx, row in df_municipios_clean.iterrows():
    province = row['PROVINCIA']
    color = province_colors.get(province, '#808080')
    
    # Create detailed popup
    popup_content = f"""
    <div style="width: 280px;">
        <h4 style="color: {color};"><b>{row['MUNICIPIO']}</b></h4>
        <hr>
        <table style="width:100%; font-size:12px;">
            <tr><td><b>Province:</b></td><td>{row['PROVINCIA']}</td></tr>
            <tr><td><b>Mayor:</b></td><td>{row['NOMBRE DEL ALCALDE']}</td></tr>
            <tr><td><b>Altitude:</b></td><td>{row['ALTURA (msnm)']} m</td></tr>
            <tr><td><b>DANE Code:</b></td><td>{row['CODIGO DANE MUNICIPIO']}</td></tr>
            <tr><td><b>NIT:</b></td><td>{row['NIT ALCALDÍA']}</td></tr>
        </table>
        <br>
        <b>Contact Information:</b><br>
        <small>
        📍 {row['DIRECCIÓN']}<br>
        📞 {row['TELEFONO']}<br>
        🌐 <a href="{row['PAGINA WEB']}" target="_blank">Website</a><br>
        ✉️ {row['CORREOELECTRÓNICO'][:50]}...
        </small>
    </div>
    """
    
    # Add marker to province group
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_content, max_width=300),
        tooltip=f"{row['MUNICIPIO']} ({province})",
        icon=folium.Icon(
            color='white',
            icon_color=color,
            icon='city',
            prefix='fa'
        )
    ).add_to(province_groups[province])

# Add all province groups to map
for group in province_groups.values():
    group.add_to(cluster_map)

# Add marker clustering
from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster(
    name="Clustered View",
    overlay=True,
    control=True
).add_to(cluster_map)

# Add clustered markers
for idx, row in df_municipios_clean.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"<b>{row['MUNICIPIO']}</b><br>Province: {row['PROVINCIA']}",
        tooltip=row['MUNICIPIO']
    ).add_to(marker_cluster)

# Add layer control
folium.LayerControl().add_to(cluster_map)

# Add title
title_html = '''
<h3 align="center" style="font-size:18px; color:#2E4057; margin-top:10px;">
<b>Boyacá Municipalities by Province - Interactive Layers</b>
</h3>
'''
cluster_map.get_root().html.add_child(folium.Element(title_html))

# Display clustered map
cluster_map

In [ ]:
# 4. Statistical analysis and visualizations
# Set up the plotting style
plt.style.use('default')
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Boyacá Municipalities - Statistical Analysis', fontsize=16, fontweight='bold')

# 1. Municipalities count by province
province_counts.plot(kind='bar', ax=axes[0,0], color=plt.cm.Set3(range(len(province_counts))))
axes[0,0].set_title('Number of Municipalities by Province')
axes[0,0].set_xlabel('Province')
axes[0,0].set_ylabel('Number of Municipalities')
axes[0,0].tick_params(axis='x', rotation=45)

# 2. Altitude distribution histogram
axes[0,1].hist(df_municipios_clean['ALTURA (msnm)'], bins=20, color='skyblue', alpha=0.7, edgecolor='black')
axes[0,1].set_title('Altitude Distribution of Municipalities')
axes[0,1].set_xlabel('Altitude (meters above sea level)')
axes[0,1].set_ylabel('Number of Municipalities')
axes[0,1].grid(True, alpha=0.3)

# 3. Altitude categories pie chart
altitude_distribution.plot(kind='pie', ax=axes[1,0], autopct='%1.1f%%', startangle=90)
axes[1,0].set_title('Distribution by Altitude Categories')
axes[1,0].set_ylabel('')

# 4. Scatter plot: Latitude vs Longitude colored by altitude
scatter = axes[1,1].scatter(df_municipios_clean['longitude'], df_municipios_clean['latitude'], 
                           c=df_municipios_clean['ALTURA (msnm)'], cmap='viridis', alpha=0.7, s=50)
axes[1,1].set_title('Geographic Distribution Colored by Altitude')
axes[1,1].set_xlabel('Longitude')
axes[1,1].set_ylabel('Latitude')
plt.colorbar(scatter, ax=axes[1,1], label='Altitude (m)')

plt.tight_layout()
plt.show()

# Summary statistics table
summary_stats = df_municipios_clean.groupby('PROVINCIA').agg({
    'ALTURA (msnm)': ['count', 'mean', 'min', 'max', 'std'],
    'latitude': 'mean',
    'longitude': 'mean'
}).round(2)

summary_stats.columns = ['Count', 'Avg_Altitude', 'Min_Altitude', 'Max_Altitude', 'Std_Altitude', 'Avg_Lat', 'Avg_Lon']
print("\nSummary Statistics by Province:")
print("="*80)
print(summary_stats)